This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# [Homework 3] Classification

## 展示題：鳶尾屬花卉分類（Binary Classification）

我們蒐集到了鳶尾屬花卉的資料集（[來源](https://archive.ics.uci.edu/ml/datasets/iris)），想要從某朵花的兩個 feature，來預測這朵花的種類。

我們將這個資料集的 csv 檔讀入至一個 pandas 的 DataFrame：`df`。資料的各個 column 的意義如下：
- `petal length`：花瓣長度
- `sepal length`：花萼長度
- `type`：花的種類，有 3 種可能
    - `Iris-setosa`（山鳶尾）
    - `Iris-versicolor`（變色鳶尾）
    - `Iris-virginica`（維吉尼亞鳶尾）

現在我們要做的是 Binary Classification，目標為：
- `Iris-setosa`：label 為 `0`
- `Iris-versicolor`：label 為 `1`

In [ ]:
!pip install numpy pandas matplotlib scikit-learn

In [ ]:
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/AINTUT/code_2022/main/datasets/iris.csv",
)

print(df)

### Data Preprocessing

讀取原始資料後，我們需要先將 input x 與 output y 根據他們需要的 feature，將資料分別取出。

由於 y 的資料有包含我們不需要的目標（`Iris-virginica`），因此必須要其剔除。剔除後我們還要將所有 `Iris-setosa` 的值轉換成 `0`，`Iris-versicolor` 的值轉換成 `1`，才能給接下來的訓練使用。

In [ ]:
# Features for x.
input_features = [
    "petal length",
    "sepal length",
]
# The feature for y.
output_feature = "type"
# The labels to be predicted.
labels = [
    "Iris-setosa",
    "Iris-versicolor",
]

In [ ]:
import numpy as np

# Get rows that belong to our target labels.
data = df[df[output_feature] != "Iris-virginica"]

# Collect data values for input x.
x_data = data[input_features].to_numpy()
print(x_data)
print(x_data.shape)

In [ ]:
# Coleccte data values for output y, and convert it to the format for
# classification.
y_data = data[output_feature]
print(y_data)
y_data = np.where(y_data == labels[0], 0, 1)
print(y_data)
print(y_data.shape)

接下來，將資料分割成 training data 與 testing data，比例為 70:30。

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing data.
x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.3,
    random_state=0,
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

再來我們用 standardization 對資料做 feature scaling。

In [ ]:
from sklearn.preprocessing import StandardScaler

# Apply feature scaling on input x.
scaler = StandardScaler()
scaler.fit(x_train)
x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

### Data Visualization

這一步驟會來視覺化我們的資料！

由於我們想要同時視覺化 training data 與 testing data，因此用另外的變數把這兩部份的資料合起來。

In [ ]:
# Combine the scaled training and testing, they are only used for visualization.
x_combined_std = np.vstack((x_train_std, x_test_std))
y_combined = np.hstack((y_train, y_test))

這裡的視覺化最重要的是看各筆資料在兩個 input features 下的分佈狀態，以及每筆資料的 label，因此我們定義了 `plot_data()` 這個 function 來達到目標。另外，如果也提供 `classifier` 給這個 function的話（晚點會用到），還可以同時把分類的 decision boundary 畫出來。

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def plot_data(x, y, labels, input_features, classifier=None, resolution=0.02):
    # Setup marker generator and color map
    colors = (
        "green",
        "red",
        "lightgreen",
    )
    markers = (
        "x",
        "s",
        "o",
    )
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # Plot the decision surface if classifier exists.
    if classifier is not None:
        x1_min, x1_max = x[:, 0].min() - 1, x[:, 0].max() + 1
        x2_min, x2_max = x[:, 1].min() - 1, x[:, 1].max() + 1
        xx1, xx2 = np.meshgrid(
            np.arange(x1_min, x1_max, resolution),
            np.arange(x2_min, x2_max, resolution),
        )
        z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
        z = z.reshape(xx1.shape)
        plt.contourf(xx1, xx2, z, alpha=0.4, cmap=cmap)
        plt.xlim(xx1.min(), xx1.max())
        plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(
            x=x[y == cl, 0],
            y=x[y == cl, 1],
            alpha=0.8,
            color=cmap(idx),
            marker=markers[idx],
            label=labels[cl],
        )

    plt.xlabel(input_features[0] + "[standardalized]")
    plt.ylabel(input_features[1] + "[standardalized]")
    plt.legend(loc="upper left")
    plt.tight_layout()

    plt.show()

In [ ]:
# Visualize the data.
plot_data(x_combined_std, y_combined, labels, input_features)

## Training

我們直接使用 scikit-learn 的 [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 來做分類的訓練。

首先，我們先創建一個型別為 `LogisticRegression ` 的物件，作為我們的 classifier（分類器），其中參數的意義為：
- `multi_class="multinomial"`：設定loss function 為 Cross Entropy
- `solver="sag"`：一種 Gradient Descent 的變化方法
- `C=1000.0`：Regularization 的程度，越小則代表程度越強
- `random_state=0`：由於 `sag` 有隨機性，因此我們填入固定的值讓同樣的結果可以重現

創建了 classifier 後，我們再用它 `fit()` 這個 method 來做訓練。


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a classifier by using Logistic Regression.
classifier = LogisticRegression(
    multi_class="multinomial",
    solver="sag",
    C=1000.0,
    random_state=0,
)
# Train the classifier.
classifier.fit(x_train_std, y_train)

訓練好之後，我們可以分別用 classifier 的 `predict_proba()` 與 `predict()` 來預測給定資料的機率與 label。下面是預測 testing data 前 3 筆資料的範例。

In [ ]:
# Predict the probabilities for the first three testing examples.
probabilities = classifier.predict_proba(x_test_std[:3, :])
print(probabilities)

# Predict the labels for the first three testing examples.
predictions = classifier.predict(x_test_std[:3, :])
print(predictions)

我們來視覺化訓練結果，除了資料外，我們也將 `classifier` 丟進 `plot_data()`，因此會同時把分類的 decision boundary 畫出來。

In [ ]:
# Visualize the data with classification results.
plot_data(x_combined_std, y_combined, labels, input_features, classifier=classifier)

## Evaluation

最後一部分，我們想要了解訓練的結果在 testing data 下表現如何。

這裡我們用 classifier 來預測所有 testing data 的 label。

In [ ]:
# Predict the labels for all testing examples.
y_pred = classifier.predict(x_test_std)
print(y_pred)
print(y_test)

我們想要得到預測結果的 confusion matrix，因此使用 scikit-learn 的 `confusion_matrix()` 來達成。要注意的是，Scikit-Learn 回傳的 confusion matrix 的順序如下，跟我們在投影片上的順序不一樣。

![](https://i.imgur.com/HQsqjR1.png)

In [ ]:
def plot_confusion_matrix(conf_mat):
    plt.matshow(conf_mat, cmap=plt.cm.Blues, alpha=0.3)

    for idx_true in range(conf_mat.shape[0]):
        for idx_pred in range(conf_mat.shape[0]):
            plt.text(
                x=idx_pred,
                y=idx_true,
                s=conf_mat[idx_true, idx_pred],
                va="center",
                ha="center",
            )

    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")

    plt.show()    

In [ ]:
from sklearn.metrics import confusion_matrix

# Construct the confusion matrix.
conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(conf_mat)

# Visualize the confusion matrix.
plot_confusion_matrix(conf_mat)

最後的最後，我們使用 scikit-learn 的 `precision_score`, `recall_score` 以及 `f1_score` 來計算預測結果的 Precsion, Recall 以及 F1 Score。

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Calculate precision, recall and f1 score.
precision = precision_score(y_true=y_test, y_pred=y_pred)
recall = recall_score(y_true=y_test, y_pred=y_pred)
f1 = f1_score(y_true=y_test, y_pred=y_pred)

print("Precision = {}".format(precision))
print("Recall = {}".format(recall))
print("F1 Score = {}".format(f1))

## 題組（一）：鳶尾屬花卉分類（Binary Classification）II

如果將 Binary Classification 的目標改成：
- `Iris-versicolor`：label 為 `0`
- `Iris-virginica`：label 為 `1`

請修改展示題的程式碼，並且求出訓練後的：
- Precision 為多少？（`versi_virgin_precision`）
- Recall 為多少？（`versi_virgin_recall`）
- F1 Score 為多少？（`versi_virgin_f1`）

In [ ]:
# PLEASE MODIFY CODE BELOW
versi_virgin_precision = 0.0
versi_virgin_recall = 0.0
versi_virgin_f1 = 0.0

print("Precision = {}".format(versi_virgin_precision))
print("Recall = {}".format(versi_virgin_recall))
print("F1 Score = {}".format(versi_virgin_f1))

## 題組（二）：鳶尾屬花卉分類（Multi-Class Classification）

如果我們改成 Multi-Class Classification，目標為：
- `Iris-setosa`：label 為 `0`
- `Iris-versicolor`：label 為 `1`
- `Iris-virginica`：label 為 `2`

請修改展示題的程式碼，並且求出訓練完成後：
- 若只考慮 `Iris-virginica` 的狀況下（亦即將 `Iris-virginica` 視為為 Positive，其他視為 Negative），則 Precision 為多少？（`virgin_only_precision`）
    - 提示：在使用 [precision_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) 的時候，需要加入一個參數 `average`
- 模型整體的 Macro Precision 為多少？Macro Precision 的定義如附註（`macro_precision`）
    - 提示：在使用 [precision_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) 的時候，需要加入一個參數 `average`

附註：Macro Precision 的算法

對於各個 label，先分別計算在只考慮該 label 的狀況下的 Precision 值，再把各個 label 的 Precision 做平均，得到 Macro Precision。

$Precision_{macro} = \frac{Precision_{1} + ... + Precision_{k}}{k}$

In [ ]:
# PLEASE MODIFY CODE BELOW
virgin_only_precision = 0.0
macro_precision = 0.0

print("Precision for Virginica only = {}".format(virgin_only_precision))
print("Macro Precision = {}".format(macro_precision))

## 答案上傳區

三區塊的功能分別如下：
1. 請將 `sid` 改為你的學號、`credential` 改為你收到的密碼
2. 執行上傳答案的程式，提交後你會看到這次上傳的每一題答案是否正確（`PASS` 為答對，`FAIL` 為答錯）
3. 查看你所有作業的答題狀況

In [ ]:
sid = "" # PLEASE MODIFY
credential = "" # PLEASE MODIFY

In [ ]:
import json
import requests

def submit_answers(
    sid,
    credential,
    ordinal,
    answers,
    api_url="https://aintut2022.herokuapp.com",
    api_version="v1",
):
    request_url = "{}/{}/assignments".format(api_url, api_version)

    res = requests.post(request_url, data={
        "sid": sid,
        "credential": credential,
        "ordinal": ordinal,
        "answers": answers,
    })

    data = res.json()

    if res.status_code >= 400:
        print("\x1b[31m{}\x1b[0m".format(json.dumps(data, indent=4)))
        return

    correctnesses = data["correctnesses"]
    for answer, correctness in zip(answers, correctnesses):
        passd_text = \
            "[\033[92mPASS\x1b[0m]" if correctness else "[\x1b[31mFAIL\x1b[0m]"
        print("{} Your answer: {}".format(passd_text, answer))

answers = [
    versi_virgin_precision,
    versi_virgin_recall,
    versi_virgin_f1,
    virgin_only_precision,
    macro_precision,
]

submit_answers(sid, credential, 3, answers)

In [ ]:
import json
import requests

def query_assignments(
    sid,
    credential,
    api_url="https://aintut2022.herokuapp.com",
    api_version="v1",
):
    request_url = "{}/{}/assignments".format(api_url, api_version)

    res = requests.get(request_url, params={
        "sid": sid,
        "credential": credential,
    })

    data = res.json()

    if res.status_code >= 400:
        print("\x1b[31m{}\x1b[0m".format(json.dumps(data, indent=4)))
        return

    assignments = sorted(data, key=lambda d: d["ordinal"])

    for assignment in assignments:
        ordinal = assignment["ordinal"]
        correctnesses = " ".join([
            "\033[92mPASS\x1b[0m" if c else "\x1b[31mFAIL\x1b[0m"
            for c in assignment["correctnesses"]
        ])
        print("Assignment {}: {}".format(ordinal, correctnesses))


query_assignments(sid, credential)